## Create data frame of Toronto postal codes with coordinates 

In [18]:
!pip install pgeocode
# Import libraries
import pandas as pd
import pgeocode 

In [34]:
# Get Canadian postal codes which start with M
HTMLtabs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
print(f'Total tables: {len(HTMLtabs)}')

# convert to dataframe df ignoring 'Not assigned' and 'Mississauga' values for the Borough
df = HTMLtabs[0][HTMLtabs[0].Borough != 'Not assigned']
df.drop(df.loc[df['Borough'] == 'Mississauga'].index, inplace=True)

# changed 'Not assigned' values for Neighbourhood to Borough values if any
for row in df.index: 
    if (df['Neighbourhood'][row] == 'Not assigned'):
        df['Neighbourhood'][row] = df['Borough'][row]
print(f'df Shape: {df.shape}')
df.head()

Total tables: 3
df Shape: (102, 3)


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [35]:
# adding Latitude and Longitude columns to df
df = pd.concat([df, pd.DataFrame(columns = ['Latitude', 'Longitude'])])
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,NaN,NaN
3,M4A,North York,Victoria Village,NaN,NaN
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN,NaN
5,M6A,North York,"Lawrence Manor, Lawrence Heights",NaN,NaN
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",NaN,NaN


In [36]:
# filling up Latitude and Longitude columns 
nm = pgeocode.Nominatim('ca')
for row in df.index: 
    df['Latitude'][row] = nm.query_postal_code(df['Postal Code'][row])['latitude']
    df['Longitude'][row] = nm.query_postal_code(df['Postal Code'][row])['longitude']

In [37]:
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.7545,-79.33
3,M4A,North York,Victoria Village,43.7276,-79.3148
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
165,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.383
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.7804,-79.2505
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939
